In [ ]:
import requests
topics_url='https://github.com/topics'
response = requests.get(topics_url)
print(response.text)
page_contents = response.text
print(page_contents[:1000])
with open('webpage.html','w',encoding='utf-8') as f:
    f.write(page_contents)

from bs4 import BeautifulSoup
doc = BeautifulSoup(page_contents,'html.parser')
selection_class = 'f3 lh-condensed mb-0 mt-1 Link--primary'
topic_title_tags = doc.find_all('p', {'class': selection_class})
print(topic_title_tags[:5])

desc_selector = 'f5 color-fg-muted mb-0 mt-1'
topic_desc_tags = doc.find_all('p', {'class': desc_selector})
print(topic_desc_tags[:5])
topic_title_tag0 = topic_title_tags[0]
div_tag=topic_title_tag0.parent

topic_link_tags = doc.find_all('a',{'class': 'no-underline flex-1 d-flex flex-column'})
topic0_url = "https://github.com" + topic_link_tags[0]['href']
print(topic0_url)

topic_titles = []
for tag in topic_title_tags:
    topic_titles.append(tag.text)

print(topic_titles)  
topic_descs = []
for tag in topic_desc_tags:
    topic_descs.append(tag.text.strip())
   

print(topic_descs[:5])

topic_urls=[]
base_url = "https://github.com"
for tag in topic_link_tags:
    topic_urls.append(base_url + tag['href'])
    print(topic_urls)







In [ ]:
import pandas as pd
topics_dict = {
    'title': topic_titles,
    'description': topic_descs,
    'url': topic_urls,
}
topics_df = pd.DataFrame(topics_dict)
print(topics_df)
topics_df.to_csv('topics.csv', index=None)

In [ ]:

topic_page_url = topic_urls[0]
topic_page_url
response = requests.get(topic_page_url)
print(response.text)
topic_doc = BeautifulSoup(response.text,'html.parser')
h1_selection_class = 'f3 color-fg-muted text-normal lh-condensed'
repo_tags = topic_doc.find_all('h3',{'class': h1_selection_class})
print(repo_tags)
a_tags = repo_tags[0].find_all('a')
print(a_tags[0].text.strip())
print(a_tags[1].text.strip())
base_url='https://github.com'
repo_url = base_url + a_tags[1]['href']
print(repo_url)
star_tags = topic_doc.find_all('span', {'class': 'Counter js-social-count'})
print(star_tags)
print(star_tags[0].text.strip())

def parse_star_count(stars_str):
    stars_str = stars_str.strip()
    if stars_str[-1] == 'k':
        return int(float(stars_str[:-1]) * 1000)
    return int(stars_str)

def get_repo_info(h1_tag, star_tag):
    a_tags = h1_tag.find_all('a')
    username = a_tags[0].text.strip()
    repo_name = a_tags[1].text.strip()
    repo_url = base_url + a_tags[1]['href']
    stars = parse_star_count(star_tag.text.strip())
    return username, repo_name, stars, repo_url

import os

def get_topic_page(topic_url):
    response = requests.get(topic_url)

    if response.status_code!=200:
        raise Exception('Failed to load page {}'.format(topic_url))
    
    topic_doc = BeautifulSoup(response.text,'html.parser')
    return topic_doc

def get_repo_info(h1_tag, star_tag):
    a_tags = h1_tag.find_all('a')
    username = a_tags[0].text.strip()
    repo_name = a_tags[1].text.strip()
    repo_url = base_url + a_tags[1]['href']
    stars = parse_star_count(star_tag.text.strip())
    return username, repo_name, stars, repo_url

def get_topic_repos(topic_doc):
    h1_selection_class = 'f3 color-fg-muted text-normal lh-condensed'
    repo_tags = topic_doc.find_all('h3', {'class': h1_selection_class})
    star_tags = topic_doc.find_all('span', {'class': 'Counter js-social-count'})

    topic_repos_dict = {
        'username': [],
        'repo_name': [],
        'stars': [],
        'repo_url': []
    }

    for i in range(len(repo_tags)):
        repo_info = get_repo_info(repo_tags[i], star_tags[i])
        topic_repos_dict['username'].append(repo_info[0])
        topic_repos_dict['repo_name'].append(repo_info[1])
        topic_repos_dict['stars'].append(repo_info[2])
        topic_repos_dict['repo_url'].append(repo_info[3])

    return pd.DataFrame(topic_repos_dict)


get_topic_repos(get_topic_page(topic_urls[6])).to_csv('ansible.csv', index = None)

def scrape_topic(topic_url, path):
   
    if os.path.exists(path):
        print("The file {} already exists. Skipping...".format(path))
        return
    topic_df = get_topic_repos(get_topic_page(topic_url))
    topic_df.to_csv(path, index = None)
    
def get_topic_titles(doc):
    selection_class = 'f3 lh-condensed mb-0 mt-1 Link--primary'
    topic_title_tags = doc.find_all('p', {'class': selection_class})
    topic_titles = []
    for tag in topic_title_tags:
        topic_titles.append(tag.text)
    return topic_titles
    
def get_topic_descs(doc):
    desc_selector = 'f5 color-fg-muted mb-0 mt-1'
    topic_desc_tags = doc.find_all('p', {'class': desc_selector})
    topic_descs = []
    for tag in topic_desc_tags:
        topic_descs.append(tag.text.strip())
    return topic_descs
    
def get_topic_urls(doc):
    topic_link_tags = doc.find_all('a',{'class': 'no-underline flex-1 d-flex flex-column'})
    topic_urls=[]
    base_url = "https://github.com"
    topic_urls = [base_url + tag['href'] for tag in topic_link_tags]
    return topic_urls
    
    
def scrape_topics():
    topics_url = 'https://github.com/topics'
    response = requests.get(topics_url)
    if response.status_code!=200:
        raise Exception('Failed to load page {}'.format(topics_url))
    topics_dict = {
        'title': get_topic_titles(doc),
        'description': get_topic_descs(doc),
        'url': get_topic_urls(doc)
    }
    return pd.DataFrame(topics_dict)

import os
help(os.makedirs)

def scrape_topics_repos():
    print('Scraping list of topics')
    topics_df = scrape_topics()

    os.makedirs('data', exist_ok=True)

    for index,row in topics_df.iterrows():
        print('Scraping top repositories for "{}"'.format(row['title']))
        scrape_topic(row['url'], 'data/{}.csv'.format(row['title']))
        
print(scrape_topics_repos())
# Create a Readme.md file for github repository using above python web scraping project


 



